In [1]:
f = open('conll2017/all/task1/finnish-train-low').read().splitlines()

In [2]:
from collections import defaultdict

fk = defaultdict(dict)
inti = 0
for item in f:
    stuff = item.split('\t')
    fk[inti]['source'] = stuff[0]
    fk[inti]['inf'] = stuff[1]
    stuff2 = stuff[2].split(';')
    fk[inti]['pos'] = stuff2[0]
    fk[inti]['num'] = stuff2[-1]
    fk[inti]['case'] = '-'.join(stuff2[1:-2])
    inti += 1

In [3]:
import pandas
pd = pandas.DataFrame.from_dict(fk,orient='index')
# Ignore the statements below
nouns = pd[pd['pos']=='N'].dropna(axis=1)
nouns.groupby(['num','case']).count()
pd[pd['pos']=='N'].dropna(axis=1)
nouns[(nouns['num']=='PL')].head(5)
data = pandas.read_csv('data/task1/output/source_csvs/finnish-train-low')
# del data['prop']
del data['Unnamed: 0']
pd = data[data['pos'] == 'N'].dropna(axis=1)

In [4]:
from IPython.display import display

rw3= pandas.DataFrame()
transM = pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
    sourLen = len(item['source'])
    for j,chara in enumerate(item['source']):
        charPairs.append([i,chara,j+1,(sourLen - j)*-1])
        
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','chara','lpos','rpos'])
    tempTrans = pandas.merge(tempDF, tempDF,how='outer',on='indi')
    tempTrans = tempTrans[(tempTrans['chara_x'] != tempTrans['chara_y']) | (tempTrans['lpos_x'] != tempTrans['lpos_y'])  | (tempTrans['rpos_x'] != tempTrans['rpos_y']) ]
    
    try:
        transM = transM.append(tempTrans)
    except:
        transM = pandas.DataFrame(tempTrans)
        
    try:
        rw3 = rw3.append(tempDF)
    except NameError:
        rw3 = pandas.DataFrame(tempDF)
        print(rw3)
        
rw3 = rw3.reset_index(drop=True)
rw3
transM

,indi,chara_x,lpos_x,rpos_x,chara_y,lpos_y,rpos_y
1,1,t,1,-12,a,2,-11
2,1,t,1,-12,v,3,-10
3,1,t,1,-12,a,4,-9
4,1,t,1,-12,r,5,-8
5,1,t,1,-12,a,6,-7
6,1,t,1,-12,m,7,-6
7,1,t,1,-12,e,8,-5
8,1,t,1,-12,r,9,-4
9,1,t,1,-12,k,10,-3
10,1,t,1,-12,k,11,-2


In [5]:
def lcs(SB,TB):
    S = SB
    T = TB
    m = len(S)
    n = len(T)
    
    
    globList = list()
    
    while True:
        lcs_set = set()
        lcsList = list()
        counter = [[0]*(n+1) for x in range(m+1)]
        longest = 0
        for i in range(m):
            for j in range(n):
                if S[i] == T[j]:
                    c = counter[i][j] + 1
                    counter[i+1][j+1] = c
                    if c > longest:
                        lcs_set = set()
                        longest = c
                        lcs_set.add(S[i-c+1:i+1])
                    elif c == longest:
                        lcs_set.add(S[i-c+1:i+1])
        
        lcsList = list(lcs_set)    
        globList.extend(lcsList)
        if len(lcs_set) == 0:
            break
        for item in lcsList:
            S = S.replace(item,'_',1)
            T = T.replace(item,'$',1)
            m = len(S)
            n = len(T)
    
    return [item for item in globList if len(item) > 1]    

In [6]:
rw4= pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
    maxSeq = lcs(item['source'],item['target'])
#     maxSeq = lcs('rataipingt', 'rntaiiingng')
#     print(maxSeq)

    str1,str2 = item['source'],item['target']
#     str1,str2 = 'rataipingt', 'rntaiiingng'

    for pat in maxSeq:
        str1 = str1.replace(pat,'$'*len(pat),1)
        str2 = str2.replace(pat,'_'*len(pat),1)
    print(str1, str2)
    str1Rems = [stuff for stuff in str1.split('$') if len(stuff) > 0]
    str2Rems = [stuff for stuff in str2.split('_') if len(stuff) > 0]
    for  indx,chara in enumerate(str1):
        if chara != '$':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    for  indx,chara in enumerate(str2):
        if chara != '_':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','opn','chara','lpos','rpos'])

    try:
        rw4 = rw4.append(tempDF)
    except NameError:
        rw4 = pandas.DataFrame(tempDF)
        print(rw4)

rw4 = rw4.reset_index(drop=True)
rw4

$$$$$$$$$$ki __________eiksi
$$$$$$$$$a _________oiksi
$$$$$$$$$nen _________sella
$$$$$$$$$ _________lta
$$$$$$$$$$ __________itä
$$$$$$$$$$i __________eiksi
$$$$$$$$$ _________na
$$$$$$ ______iksi
$$$$$$$$$$ __________en
$$$$$$$ _______
$$$$$ _____ssa
$$$$$$$ _______lla
$$$$$ _____sta
$$$$$ _____iksi
$$$$$$$$$$$$ta ____________nille
$$$$a ____oiksi
$$$$$$$$$$s __________delle
$$$$$$$$$$$$$$$$ki ________________eille
$$$$$ti _____illa
$$$$$$$a _______oin
$$$$$$$$$$$$ ____________en
$$$$$$ ______stä
$$$$$$$$$i _________eihin
$$$$$ _____ssa
$$$$$$$$$$$$$s _____________det
$$$$$nen _____sista
$$$$$$$$$ _________eille
$$$$$$$$$$$$$$ka ______________oilta
$$$$$$s ______ksenä
$$$$$ _____an
$$$$$$$$$$$$$$$$ ________________lle
$$$$$$$$$ _________
$$$$$$$$$$$ä ___________iin
$$$$$$$ _______ksi
$$$$$$$a _______oina
$$$$$$$$$$$$$$$i _______________eja
$$$$$$$$$$$$ ____________a
$$$$$$$$$$mas __________pailla
$$$$$$$$$$ __________ssä
$$$$$$i ______ea
$$$$ ____
$$$$$$$$$$$$s ____________ksissa
$$

,indi,opn,chara,lpos,rpos
0,1.0,del,k,11.0,-2.0
1,1.0,del,i,12.0,-1.0
2,1.0,del,e,11.0,-2.0
3,1.0,del,i,12.0,-1.0
4,1.0,del,k,13.0,1.0
5,1.0,del,s,14.0,2.0
6,1.0,del,i,15.0,3.0
7,7.0,del,a,10.0,-1.0
8,7.0,del,o,10.0,-1.0
9,7.0,del,i,11.0,1.0
